## The goal
The **goal** of our experiments is to determine the coefficient $a_{t_i t_j}$ of impact of type ${t_i}$ on type ${t_j}$.

We run the experiment and we calculate the coefficient. Then we run it again, and we get a slightly different result. We can proceed with that procedure and think of it as sampling a **distribution** of coefficients.

### Is this distribution normal?
This distribution is not necessarily normal, but we suspect it to be normal, because we use average latency/throughput from some time intervals to calculate our coefficient.

Luckily, this can be checked with large enough sample: https://en.wikipedia.org/wiki/Normality_test

### How to determine one coefficient from a list samples?
This is an important conceptual question. We want a coefficient that reflects the impact of two types best.

One of the possible aproaches is to simply take the **average coefficient**:
* The average coefficient is the good for most number of cases - in total, we gain the most.
* We already use average values to calculate the coefficient in every experiment

### How many samples are necessary?
We want to be as accurate as possible, but each experiment takes much time to run.

If we use average coefficient, then we can use interval estimation to calculate the confidence interval with some confidence level, using following formula:

$ P\left({\overline {X}}-u_{\alpha }{\frac {\sigma }{\sqrt {n}}}<m<{\overline {X}}+u_{\alpha }{\frac {\sigma }{\sqrt {n}}}\right)=1-\alpha $

where:
* $n$ – sample size,
* $\overline {X}$ – average of the sample,
* $\sigma$  – standard deviation of the sample,
* $u_{\alpha }$ – a statistic satysfying $P(-u_{\alpha }<U<u_{\alpha })=1-\alpha$, where $U$ is of $N(0,1)$

Find more: https://en.wikipedia.org/wiki/Confidence_interval

In [2]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, h


def print_confidence_intervals(measurements):
    confidences = [0.8, 0.9, 0.95, 0.99, 0.999]
    print(f'Measurements: {measurements}')
    
    for confidence in confidences:
        m, h = mean_confidence_interval(measurements, confidence=confidence)
        print(f'Mean is between {m-h:.4} and {m+h:.4} (±{h/m*100:.4}%) with confidence {confidence}')
    print('')

        
measurements = [0.512, 0.534, 0.491]
print_confidence_intervals(measurements)


measurements = [0.512, 0.534, 0.491, 0.522]
print_confidence_intervals(measurements)
    
measurements = [0.512, 0.534, 0.491, 0.522, 0.481]
print_confidence_intervals(measurements)

Measurements: [0.512, 0.534, 0.491]
Mean is between 0.4889 and 0.5357 (±4.569%) with confidence 0.8
Mean is between 0.4761 and 0.5486 (±7.075%) with confidence 0.9
Mean is between 0.4589 and 0.5657 (±10.43%) with confidence 0.95
Mean is between 0.3891 and 0.6355 (±24.05%) with confidence 0.99
Mean is between 0.1201 and 0.9046 (±76.57%) with confidence 0.999

Measurements: [0.512, 0.534, 0.491, 0.522]
Mean is between 0.4998 and 0.5297 (±2.897%) with confidence 0.8
Mean is between 0.4933 and 0.5362 (±4.163%) with confidence 0.9
Mean is between 0.4858 and 0.5437 (±5.629%) with confidence 0.95
Mean is between 0.4616 and 0.5679 (±10.33%) with confidence 0.99
Mean is between 0.3971 and 0.6324 (±22.86%) with confidence 0.999

Measurements: [0.512, 0.534, 0.491, 0.522, 0.481]
Mean is between 0.493 and 0.523 (±2.946%) with confidence 0.8
Mean is between 0.4872 and 0.5288 (±4.097%) with confidence 0.9
Mean is between 0.4809 and 0.5351 (±5.335%) with confidence 0.95
Mean is between 0.4631 and 0.5

We can see that the interval is narrowing down significantly as size of sample increases.

Here we have some linear regression coefficients obtained by adding instances of `redis_ycsb` on `naan`: `[0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]`. Let's see how our confidence intervals look like:

In [3]:
measurements = [0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]

for i in range(0,3):
    print_confidence_intervals(measurements[:i+2])

Measurements: [0.01432621438691767, 0.014196753917670415]
Mean is between 0.01406 and 0.01446 (±1.397%) with confidence 0.8
Mean is between 0.01385 and 0.01467 (±2.866%) with confidence 0.9
Mean is between 0.01344 and 0.01508 (±5.767%) with confidence 0.95
Mean is between 0.01014 and 0.01838 (±28.89%) with confidence 0.99
Mean is between -0.02695 and 0.05547 (±288.9%) with confidence 0.999

Measurements: [0.01432621438691767, 0.014196753917670415, 0.014124212515962489]
Mean is between 0.0141 and 0.01433 (±0.7836%) with confidence 0.8
Mean is between 0.01404 and 0.01439 (±1.214%) with confidence 0.9
Mean is between 0.01396 and 0.01447 (±1.788%) with confidence 0.95
Mean is between 0.01363 and 0.0148 (±4.125%) with confidence 0.99
Mean is between 0.01235 and 0.01608 (±13.13%) with confidence 0.999

Measurements: [0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]
Mean is between 0.01415 and 0.01429 (±0.4814%) with confidence 0.8
Mean is between 0.01412

We can see that the interval is narrowing down significantly as size of sample increases.

Here we have some linear regression coefficients obtained by adding instances of `redis_ycsb` on `naan`:
```python
measurements = [0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]
```
Let's see how our confidence intervals look like:

In [4]:
measurements = [0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]

for i in range(0,3):
    print_confidence_intervals(measurements[:i+2])

Measurements: [0.01432621438691767, 0.014196753917670415]
Mean is between 0.01406 and 0.01446 (±1.397%) with confidence 0.8
Mean is between 0.01385 and 0.01467 (±2.866%) with confidence 0.9
Mean is between 0.01344 and 0.01508 (±5.767%) with confidence 0.95
Mean is between 0.01014 and 0.01838 (±28.89%) with confidence 0.99
Mean is between -0.02695 and 0.05547 (±288.9%) with confidence 0.999

Measurements: [0.01432621438691767, 0.014196753917670415, 0.014124212515962489]
Mean is between 0.0141 and 0.01433 (±0.7836%) with confidence 0.8
Mean is between 0.01404 and 0.01439 (±1.214%) with confidence 0.9
Mean is between 0.01396 and 0.01447 (±1.788%) with confidence 0.95
Mean is between 0.01363 and 0.0148 (±4.125%) with confidence 0.99
Mean is between 0.01235 and 0.01608 (±13.13%) with confidence 0.999

Measurements: [0.01432621438691767, 0.014196753917670415, 0.014124212515962489, 0.014220166651701715]
Mean is between 0.01415 and 0.01429 (±0.4814%) with confidence 0.8
Mean is between 0.01412

Here we have some coefficients obtained by adding instances of `redis_ycsb` on `baati`.

```python
measurements = [0.006191, 0.004276, 0.006411, 0.006523, 0.006287]
```

This time we used `statsmodels` to calculate them.

In [5]:
measurements = [0.006191, 0.006411, 0.006523, 0.006287]

for i in range(0,3):
    print_confidence_intervals(measurements[:i+2])

Measurements: [0.006191, 0.006411]
Mean is between 0.005962 and 0.00664 (±5.373%) with confidence 0.8
Mean is between 0.005606 and 0.006996 (±11.02%) with confidence 0.9
Mean is between 0.004903 and 0.007699 (±22.18%) with confidence 0.95
Mean is between -0.0007012 and 0.0133 (±111.1%) with confidence 0.99
Mean is between -0.06373 and 0.07633 (±1.111e+03%) with confidence 0.999

Measurements: [0.006191, 0.006411, 0.006523]
Mean is between 0.006191 and 0.006559 (±2.884%) with confidence 0.8
Mean is between 0.00609 and 0.00666 (±4.467%) with confidence 0.9
Mean is between 0.005955 and 0.006795 (±6.582%) with confidence 0.95
Mean is between 0.005407 and 0.007343 (±15.18%) with confidence 0.99
Mean is between 0.003294 and 0.009456 (±48.34%) with confidence 0.999

Measurements: [0.006191, 0.006411, 0.006523, 0.006287]
Mean is between 0.006234 and 0.006472 (±1.866%) with confidence 0.8
Mean is between 0.006183 and 0.006523 (±2.681%) with confidence 0.9
Mean is between 0.006123 and 0.006583 (

### redis_ycsb_d vs redis_ycsb_d from `puri`.

In [6]:
# measurements_a = [0.586740, 0.569640, 0.569640, 0.547871]
measurements_d = [0.581130, 0.593587, 0.564477, 0.565992, 0.567540, 0.573274, 0.569143, 0.578121]

for i in range(len(measurements_d) - 5, len(measurements_d) + 1):
    print_confidence_intervals(measurements_d[:i])

Measurements: [0.58113, 0.593587, 0.564477]
Mean is between 0.5638 and 0.5956 (±2.743%) with confidence 0.8
Mean is between 0.5551 and 0.6044 (±4.247%) with confidence 0.9
Mean is between 0.5434 and 0.616 (±6.258%) with confidence 0.95
Mean is between 0.496 and 0.6634 (±14.44%) with confidence 0.99
Mean is between 0.3133 and 0.8462 (±45.96%) with confidence 0.999

Measurements: [0.58113, 0.593587, 0.564477, 0.565992]
Mean is between 0.565 and 0.5876 (±1.956%) with confidence 0.8
Mean is between 0.5601 and 0.5925 (±2.81%) with confidence 0.9
Mean is between 0.5544 and 0.5982 (±3.8%) with confidence 0.95
Mean is between 0.5361 and 0.6165 (±6.974%) with confidence 0.99
Mean is between 0.4874 and 0.6652 (±15.43%) with confidence 0.999

Measurements: [0.58113, 0.593587, 0.564477, 0.565992, 0.56754]
Mean is between 0.5659 and 0.5831 (±1.497%) with confidence 0.8
Mean is between 0.5626 and 0.5865 (±2.082%) with confidence 0.9
Mean is between 0.559 and 0.5901 (±2.711%) with confidence 0.95
Mea

In [9]:
def boot(m):
    for i in range(len(m) - 5, len(m) + 1):
        print(f'Measurements: {m[:i]}')
        for a in [0.8, 0.9, 0.95, 0.99, 0.999]:
            res = bootstrap((m[:i],), np.mean, confidence_level=a, n_resamples=99999)
            low = res.confidence_interval.low
            high = res.confidence_interval.high
            h = high - low
            me = (low + high) / 2
            p = h / me * 100
            print(f'Mean is {me:.4f} (±{p:.3f}%) with confidence {a}')

ImportError: cannot import name 'bootstrap' from 'scipy.stats' (/home/godul/.local/lib/python3.7/site-packages/scipy/stats/__init__.py)